In [ ]:
from imports_scgen_local import * #imports_scgen_local have to be in same folder

## To Do update with new DGE Table

In [ ]:
pre = 'MH69'
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                              pre = pre,
                                                                              add_path = True)

In [ ]:
_,base_table_path_MH68,_,_ = h.return_local_paths(drive = drive,pre = 'MH68',add_path = True)

In [ ]:
_,base_table_path_M25_v2,_,_ = h.return_local_paths(drive = drive,pre = "M25_v2",add_path = False)
_,base_table_path_H25_v2,_,_ = h.return_local_paths(drive = drive,pre = "H25_v2",add_path = False)

In [ ]:
df_dge_M = pd.read_csv(os.path.join(base_table_path_M25_v2,'M25_v2_dge_tab_edgeR.csv'),index_col=0)
df_dge_H = pd.read_csv(os.path.join(base_table_path_H25_v2,'H25_v2_dge_tab_edgeR.csv'),index_col=0)

In [ ]:
df_pert_all = pd.read_csv(os.path.join(base_table_path_MH68,'MH68df_pert_all.csv'))

In [ ]:
df_pert_all['human_gene_name'] = df_pert_all['gene'].str.split('--').str[0]
df_pert_all['cyno_gene_name'] = df_pert_all['gene'].str.split('--').str[1]

In [ ]:
comps_DGE = ['timepoints06hr', 'timepoints24hr']

In [ ]:
df_dge_H['celltype'] =df_dge_H['celltype'].str.replace('CD4_T','CD4 T')
df_dge_H['celltype'] =df_dge_H['celltype'].str.replace('CD8_T','CD8 T')
df_dge_H['celltype'] =df_dge_H['celltype'].str.replace('other_T','other T')

In [ ]:
df_dge_M['celltype'] =df_dge_M['celltype'].str.replace('CD4_T','CD4 T')
df_dge_M['celltype'] =df_dge_M['celltype'].str.replace('CD8_T','CD8 T')
df_dge_M['celltype'] =df_dge_M['celltype'].str.replace('other_T','other T')

In [ ]:
columns = ['species','celltype','cond_diff_exp' ,'spearman_correlation', 'spearman_pval','fraction_top_100']
df_spearman = pd.DataFrame(columns=columns)

for celltype in ['CD4 T','CD8 T','B']:
    for comp_DGE in comps_DGE:
        #human
        df_pert_all_ct = df_pert_all[df_pert_all['celltype']== celltype]
        df_dge_H_ct = df_dge_H[(df_dge_H['celltype'] == celltype)&(df_dge_H['comparison.vs.00hr'] == comp_DGE)]
        df_dge_H_ct = df_dge_H_ct.rename(columns={'genes': 'human_gene_name'})
        df_dge_H_ct = df_dge_H_ct[df_dge_H_ct["FDR"] <= 1]
        intersect_genes = set(list(df_pert_all_ct['human_gene_name'])) & set(list(df_dge_H_ct ['human_gene_name'])) 
        df_pert_all_ct = df_pert_all_ct[df_pert_all_ct['human_gene_name'].isin(intersect_genes)]
        df_dge_H_ct =df_dge_H_ct[df_dge_H_ct['human_gene_name'].isin(intersect_genes)]
        df_pert_all_ct['rank_VAE'] = df_pert_all_ct['mean_perturbation_effect'].rank(method='dense', ascending=False).astype(int)
        df_dge_H_ct['rank_diff_exp'] = df_dge_H_ct['PValue'].rank(method='dense', ascending=True).astype(int)
        merged_df_H = pd.merge(df_pert_all_ct,df_dge_H_ct, on='human_gene_name', how='inner')
        merged_df_H.to_csv(os.path.join(base_table_path,pre + '_Human_' + celltype +'_' + comp_DGE + '.csv'))
        spearman_res_H = spearmanr(merged_df_H['rank_diff_exp'], merged_df_H['rank_VAE'])
        fraction_top_100_H = sum(merged_df_H[merged_df_H['rank_diff_exp']<=100]['rank_VAE'] <= 100)/len(merged_df_H[merged_df_H['rank_diff_exp']<=100])
        row_to_add = ['human',celltype,comp_DGE,spearman_res_H.correlation,spearman_res_H.pvalue,fraction_top_100_H]
        df_spearman = df_spearman.append(pd.Series(row_to_add, index=df_spearman.columns), ignore_index=True)
       

        #cyno
        df_pert_all_ct = df_pert_all[df_pert_all['celltype']== celltype]
        df_dge_M_ct = df_dge_M[(df_dge_M['celltype'] == celltype)&(df_dge_M['comparison.vs.00hr'] == comp_DGE)]
        df_dge_M_ct = df_dge_M_ct.rename(columns={'genes': 'human_gene_name'})
        df_dge_M_ct = df_dge_M_ct[df_dge_M_ct["FDR"] <= 1]
        intersect_genes = set(list(df_pert_all_ct['human_gene_name'])) & set(list(df_dge_M_ct ['human_gene_name'])) 
        df_pert_all_ct = df_pert_all_ct[df_pert_all_ct['human_gene_name'].isin(intersect_genes)]
        df_dge_M_ct =df_dge_M_ct[df_dge_M_ct['human_gene_name'].isin(intersect_genes)]
        df_pert_all_ct['rank_VAE'] = df_pert_all_ct['mean_perturbation_effect'].rank(method='dense', ascending=False).astype(int)
        df_dge_M_ct['rank_diff_exp'] = df_dge_M_ct['PValue'].rank(method='dense', ascending=True).astype(int)
        merged_df_M = pd.merge(df_pert_all_ct,df_dge_M_ct, on='human_gene_name', how='inner')
        merged_df_M.to_csv(os.path.join(base_table_path,pre + '_cyno_' + celltype +'_' + comp_DGE + '.csv'))
        spearman_res_M = spearmanr(merged_df_M['rank_diff_exp'], merged_df_M['rank_VAE'])
        fraction_top_100_M = sum(merged_df_M[merged_df_M['rank_diff_exp']<=100]['rank_VAE'] <= 100)/len(merged_df_M[merged_df_M['rank_diff_exp']<=100])
        row_to_add = ['cyno',celltype,comp_DGE,spearman_res_M.correlation,spearman_res_M.pvalue,fraction_top_100_M]
        df_spearman = df_spearman.append(pd.Series(row_to_add, index=df_spearman.columns), ignore_index=True)

In [ ]:
df_spearman['cond_diff_exp+species'] = df_spearman['cond_diff_exp'].astype(str) + df_spearman['species'].astype(str)
df_spearman['abs_spearman_correlation'] = np.abs(df_spearman['spearman_correlation']) 
df_spearman['signum'] = np.sign(df_spearman['spearman_correlation'])

In [ ]:
df_spearman.to_csv(os.path.join(base_table_path,pre + '_df_spearman.csv'))

In [ ]:
plt.figure
scatter = sns.scatterplot(
    data=df_spearman,
    y='cond_diff_exp+species',
    x='celltype',
    size='abs_spearman_correlation',
    sizes=(10, 800),  # Adjust size range
    edgecolor='w',
    style='signum',
    markers={1: '^', -1: 'v'},
    hue='spearman_pval',
    palette='Purples_r'
    #legend=True  # Hide the automatic legend
    )
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.savefig(os.path.join(base_plots_path,pre + '_correlation_VAE_DGE_rank.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)